In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/Shareddrives/EECS\ 442\ -\ Project/eecs442-proj
!git config --global user.email "wzahm@umich.edu"
#!git config --global user.name "Wyler Zahm"
!git status
!git pull
!git add Executor.ipynb
!git commit -m "Adding executor noteboook."
!git push
#!git push

/content/drive/Shareddrives/EECS 442 - Project/eecs442-proj
^C


In [ ]:
# Get data
!wget http://diode-dataset.s3.amazonaws.com/val.tar.gz